In [1]:
%%time

#Modded from qutip to allow parallelism
import numpy as np
import scipy.linalg as la
from numpy import angle, pi
from qutip import Qobj, propagator

def floquet_modes_mod(H, T, args=None, parallel=False, sort=False, U=None):
    """
    Calculate the initial Floquet modes Phi_alpha(0) for a driven system with
    period T.

    Returns a list of :class:`qutip.qobj` instances representing the Floquet
    modes and a list of corresponding quasienergies, sorted by increasing
    quasienergy in the interval [-pi/T, pi/T]. The optional parameter `sort`
    decides if the output is to be sorted in increasing quasienergies or not.

    Parameters
    ----------

    H : :class:`qutip.qobj`
        system Hamiltonian, time-dependent with period `T`

    args : dictionary
        dictionary with variables required to evaluate H

    T : float
        The period of the time-dependence of the hamiltonian. The default value
        'None' indicates that the 'tlist' spans a single period of the driving.

    U : :class:`qutip.qobj`
        The propagator for the time-dependent Hamiltonian with period `T`.
        If U is `None` (default), it will be calculated from the Hamiltonian
        `H` using :func:`qutip.propagator.propagator`.

    Returns
    -------

    output : list of kets, list of quasi energies

        Two lists: the Floquet modes as kets and the quasi energies.

    """
    if 'opts' in args:
        options = args['opts']
    else:
        options = Options()
        options.rhs_reuse = True
        rhs_clear() 
    
    if U is None:
        # get the unitary propagator
        U = propagator(H, T, [], args, parallel=parallel, progressbar=True, options=options)
    
    # find the eigenstates for the propagator
    evals, evecs = la.eig(U.full())

    eargs = angle(evals)

    # make sure that the phase is in the interval [-pi, pi], so that
    # the quasi energy is in the interval [-pi/T, pi/T] where T is the
    # period of the driving.  eargs += (eargs <= -2*pi) * (2*pi) +
    # (eargs > 0) * (-2*pi)
    eargs += (eargs <= -pi) * (2 * pi) + (eargs > pi) * (-2 * pi)
    e_quasi = -eargs / T

    # sort by the quasi energy
    if sort:
        order = np.argsort(-e_quasi)
    else:
        order = list(range(len(evals)))

    # prepare a list of kets for the floquet states
    new_dims = [U.dims[0], [1] * len(U.dims[0])]
    new_shape = [U.shape[0], 1]
    kets_order = [Qobj(np.matrix(evecs[:, o]).T,
                       dims=new_dims, shape=new_shape) for o in order]

    return kets_order, e_quasi[order]

CPU times: user 399 ms, sys: 25.1 ms, total: 424 ms
Wall time: 428 ms


In [2]:
%%time
import numpy as np
from qutip import tensor, sigmax, sigmay, sigmaz, qeye, mesolve, expect

def drive(t, args):
    h0 = args['h0']
    h = args['h']
    w = args['omega']
    return h0 + h * np.cos(w * t)

def get_sx(N):
    si = qeye(2)
    sx = sigmax()
    sx_list = []
    for n in range(N):
        op_list = []
        for m in range(N):
            op_list.append(si)

        op_list[n] = sx
        sx_list.append(tensor(op_list))
    SX = 0.0    
    for n in range(N):
        SX += sx_list[n] 
    return SX    

def get_density_operators(N):
    si = qeye(2)
    sy = sigmay()
    sz = sigmaz()
    s_plus = 0.5 * (sy + (1j)*sz)
    s_minus = 0.5 * (sy - (1j)*sz)
    
    density_op = s_plus * s_minus
    
    density_list = []
    
    for n in range(N):
        op_list = []
        for m in range(N):
            op_list.append(si)

        op_list[n] = density_op
        operator = tensor(op_list)
        density_list.append(operator/operator.norm())
    return density_list


def get_hamiltonian_zzx(N, args,jmat = None, **kwargs):
    si = qeye(2)
    
    sx = sigmax()
    sz = sigmaz()

    sz_list = []

    for n in range(N):
        op_list = []
        for m in range(N):
            op_list.append(si)

        op_list[n] = sz
        sz_list.append(tensor(op_list))
    # construct the hamiltonian
    H0 = 0.0

    # interaction terms
    for i in range(N-1):
        for j in range(i+1, N):
            H0 += jmat[i,j] * sz_list[i] * sz_list[j]
            
    H1 = get_sx(N)
    H = [H0,[H1,drive]]  
    return H

def get_observables_zzx(N, args,tlist=np.linspace(0,1,100),jmat = None, e_ops=None, **kwargs):
    if jmat is None:
        jmat = np.diag(np.ones(N),1)
    T = 2 * np.pi/args['omega']
    H = get_hamiltonian_zzx(N, args,jmat = jmat, **kwargs)
    opts = args['opts']
    SX = get_sx(N)
    _, psi0 = SX.groundstate()
    return mesolve(H, psi0,tlist ,c_ops = [], e_ops=e_ops ,args=args, options=opts)
    

def get_floquet_zzx(N, args,jmat = None, **kwargs):
    if jmat is None:
        jmat = np.diag(np.ones(N),1)
    T = 2 * np.pi/args['omega']
    return floquet_modes_mod(get_hamiltonian_zzx(N, args,jmat = jmat, **kwargs), T, args, **kwargs)

def get_ipr(N, args, **kwargs):
    f_states, _ = get_floquet_zzx(N,args,**kwargs)
    rhos = get_density_operators(N)
    ipr = np.array([np.sum([np.abs(expect(rho_i, state))**2 for rho_i in rhos]) for state in f_states])
    return ipr

CPU times: user 67 µs, sys: 23 µs, total: 90 µs
Wall time: 81.8 µs


In [3]:
%%time
import numpy as np
from scipy.special import jn_zeros
from qutip import Options
import h5py

filename='w_vs_iprs_frz_run3.hdf5'
f = h5py.File(filename, 'a')
N=12
w_min=14.1
w_max=21
n_omegas = 33
nprocs=48


jmat = np.full((N, N), 2/(N-1))
np.fill_diagonal(jmat,0)

opts = Options(nsteps=1e6, num_cpus=nprocs, openmp_threads=1, atol=1e-12, rtol=1e-14)
omegas = np.linspace(w_min, w_max, n_omegas)

dset = f.create_dataset("iprs", (n_omegas, 2**N))

dset.attrs['N']=N
dset.attrs['jmat']=jmat
h0 = np.pi/10
eta = jn_zeros(0,4)[-1]
ampls = eta * omegas/4.0
parameters = np.vstack((ampls,omegas)).T
dset.attrs['h-w'] = parameters
f.close()

for i,param in enumerate(parameters): 
    h,omega = param
    p = {"h0":h0, "omega":omega, "h":h, "opts":opts}
    with h5py.File(filename, 'a') as fp:
        dset = fp['iprs']
        dset[i] = get_ipr(N,p,jmat=jmat,parallel=True, sort=True)

CPU times: user 978 ms, sys: 1.53 s, total: 2.51 s
Wall time: 1.12 s
